## Reference
* lovit님의 soyspacing tutorial을 따라하며 공부한 자료임을 밝혀둡니다.

In [1]:
from soyspacing.countbase import RuleDict, CountSpace
import soyspacing
print(soyspacing.__version__)

1.0.17


### read data and train model

In [8]:
with open('./134963_norm.txt', encoding='utf-8') as f:
    lines = f.readlines()
lines[0:10]

['시사회에서 보고왔습니다동화와 재즈뮤지컬의 만남 지루하지않고 재밌습니다\t9\n',
 '사랑과 꿈 그 흐름의 아름다움을 음악과 영상으로 최대한 담아놓았다 배우들 연기는 두말할것없고\t10\n',
 '지금껏 영화 평가 해본 적이 없는데 진짜 최고네요 색감 스토리 음악 연기 모두ㅜㅜ최고입니다\t10\n',
 '방금 시사회 보고 왔어요 배우들 매력이 눈을 뗄 수가 없게 만드네요 한편의 그림 같은 장면들도 많고 음악과 춤이 눈과 귀를 사로 잡았어요 한번 더 보고 싶네요\t10\n',
 '초반부터 끝까지 재미있게 잘보다가 결말에서 고국마 왕창먹음 힐링 받는 느낌들다가 막판에 기분 잡쳤습니다 마치 감독이 하고싶은 말은 너희들이 원하는 결말은 이거지 하지만 현실은 이거다 라고 말하고 싶었나보군요\t1\n',
 '시사회 보고 왔어요 꿈과 사랑에 관한 이야기인데 뭔가 진한 여운이 남는 영화예요\t9\n',
 '황홀하고 따뜻한 꿈이었어요 imax로 또 보려합니다 좋은 영화 시사해주셔서 감사해요\t10\n',
 '엠마스톤의 노래 솜씨도 보겠군\t7\n',
 '영화는 꼭 가서 보시길 바랍니다 소장용 가지 100 스페셜패키지 매진전 예매하느라 예매열리기전에 하루종일 CGV앱을 들여다본건 처음이네요\t10\n',
 '엠마는 코미디영화 배우 아님 코미디 배우가 멜로를 ㅋㅋㅋ 목소리가 전혀 안어울리는데\t1\n']

In [4]:
corpus_fname = './134963_norm.txt'
model = CountSpace()
model.train(corpus_fname)

all tags length = 694236 --> 57795, (num_doc = 15602)

### 띄어쓰기 교정 함수 적용
* force_abs_threshold: 점수의 절대값이 이 수준 이상이면 최고점이 아니더라도 즉각 태깅
* nonspace_threshold: 이 점수 이하일 때만 0으로 태깅
* space_threshold: 이 점수 이상일 때만 1로 태깅
* min_count: L, C, R 각각의 feature 빈도수가 min_count 이하이면 불확실한 정보로 판단, 띄어쓰기 계산 시 무시
* verbose: iteration마다 띄어쓰기가 어떻게 되고 있는지 확인
* rules: 점수와 관계 없이 반드시 태깅을 먼저 할 (chars,tags)

In [5]:
verbose=False
mc = 10  # min_count
ft = 0.3 # force_abs_threshold
nt =-0.3 # nonspace_threshold
st = 0.3 # space_threshold


sent = '이건진짜좋은영화 라라랜드진짜좋은영화'

sent_corrected, tags = model.correct(
    sent,
    verbose=verbose,
    force_abs_threshold=ft,
    nonspace_threshold=nt,
    space_threshold=st,
    min_count=mc
)

print('before: %s' % sent)
print('after : %s' % sent_corrected)

before: 이건진짜좋은영화 라라랜드진짜좋은영화
after : 이건 진짜 좋은 영화 라라랜드진짜 좋은 영화


* model.correct는 input으로 띄어쓰기를 할 text가 입력됨.
* model.correct의 결과물은 띄어쓰기가 수정된 문장과 각 글자의 태그가 출력됨.
* 띄어쓰기를 해야할지 말지 모호한 경우에는 None으로 표시되며 띄어쓰기 수정 시에 None은 0과 함께 띄지 않음.

In [6]:
tags

[0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, None, None, 1, 0, 1, 0, 1]

근데 위에 after을 살펴보면 '라라랜드진짜'의 띄어쓰기가 잘 안된것을 확인할 수 있다.<br>
이와 같은 경우, 진짜는 무조건 띄어써야하는데 이를 강제할 수 있다.

* RuleDict 안에는 chars:tag가 들어있다.<br>
 * chars: str type
 * tags: tuple type
 
* rules는 룰 단어 길이보다 1이 더 긴 태그 정보가 들어가야 한다.
 * 룰 단어 앞에를 띄우는지 (1), 안 띄우는지 (1)을 표시해야 함.
 * 불확실한 경우에는 룰 파일에 ?라고 기록하면 None으로 바꿔서 로딩됨.
   * ex) 지수가 ?001 -> "지수가": (None, 0, 0, 1)

In [10]:
from soyspacing.countbase import RuleDict

with open('./space_rules.txt', encoding='utf-8') as f:
    for doc in f:
        print(doc.strip())

가령	101
진짜	101
가게는	1001
가게로	1001
가게야	1001


In [16]:
# rule_dict의 형태는 str:tag의 형태
rule_fnames = ['./space_rules.txt']

rule_dict = RuleDict(rule_fnames)
rule_dict.rule_dict

{'가령': (1, 0, 1),
 '진짜': (1, 0, 1),
 '가게는': (1, 0, 0, 1),
 '가게로': (1, 0, 0, 1),
 '가게야': (1, 0, 0, 1)}

In [21]:
print('rule_dict을 적용하기 이전 결과 입니다.')
print(model.correct(sent, mc, ft, nt, st, verbose=verbose))
print('\n')
print('rule_dict을 적용한 후의 결과 입니다.')
print(model.correct(sent, mc, ft, nt, st, rule_dict, verbose=verbose))

rule_dict을 적용하기 이전 결과 입니다.
('이건 진짜 좋은 영화 라라랜드진짜 좋은 영화', [0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, None, None, 1, 0, 1, 0, 1])


rule_dict을 적용한 후의 결과 입니다.
('이건 진짜 좋은 영화 라라랜드 진짜 좋은 영화', [0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1])


## 네이버 영화 리뷰로 training 해보기

In [22]:
corpus_fname = 'C:/Users/sbh0613/Desktop/NLP/ratings_train.txt'
model = CountSpace()
model.train(corpus_fname)

all tags length = 139793 --> 139705, (num_doc = 150000))

In [31]:
verbose=False
mc = 10  # min_count
ft = 0.3 # force_abs_threshold
nt =-0.3 # nonspace_threshold
st = 0.3 # space_threshold


sent = '이건정말진짜최악의영화'

sent_corrected, tags = model.correct(
    sent,
    verbose=verbose,
    force_abs_threshold=ft,
    nonspace_threshold=nt,
    space_threshold=st,
    min_count=mc
)

print('before: %s' % sent)
print('after : %s' % sent_corrected)

before: 이건정말진짜최악의영화
after : 이건 정말진짜 최악의 영화


In [32]:
print('rule_dict을 적용하기 이전 결과 입니다.')
print(model.correct(sent, mc, ft, nt, st, verbose=verbose))
print('\n')
print('rule_dict을 적용한 후의 결과 입니다.')
print(model.correct(sent, mc, ft, nt, st, rule_dict, verbose=verbose))

rule_dict을 적용하기 이전 결과 입니다.
('이건 정말진짜 최악의 영화', [0, 1, 0, None, None, 1, 0, 0, 1, 0, 1])


rule_dict을 적용한 후의 결과 입니다.
('이건 정말 진짜 최악의 영화', [0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1])
